# Examples of using Simulations and RealNVP

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [11]:
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.animation as animation

sys.path.append("../core")
sys.path.append("../core/simulations")


from losses import getLoss
from trainer import Trainer
from optimizers import getOpt
from network_base import RealNVP
from example_data import gen_double_moon_samples
from network_logging import *
from simulations.visuals import *
from simulations.simulations import *

plt.style.use("ggplot")

## Crescent Moon Example

A basic example of the RealNVP network. The data is generated by mixing samples from two similar distributions such that the shape looks like the following:

<img src="images/doublemoon.png" width=450 height=175>


We want the network to learn how to generate samples from a distribution built from neural network layers. The network starts with being able to generate a simple gaussian distribution and transforms the network parameters so that it generates a more complex distribution.


By running the `crescent_moon_example` we generate that network by training over samples generated from the original distribution. 

In [ ]:
def crescent_moon_example():
    data = gen_double_moon_samples(10000)
    loss = getLoss().ml_loss()
    opt = getOpt().rmsprop(1e-4)
    model = RealNVP(loss, opt, model_name='double_moon')
    model = LogLoss(model)
    model = LogTargetPlot(model, xlim=[-5,5], ylim=[-5,5])
    model = Checkpointing(model)
    trainer = Trainer(model, data)
    trainer.train(20)
    return trainer

In [ ]:
trained = crescent_moon_example()

In [ ]:
samples = trained.model.forward_sample(10000).numpy().T
plt.figure(figsize=(9,6))
plt.title("Samples from Double Moon Trained Model")
plt.scatter(samples[0], samples[1], s=1.15, c='white', cmap='jet')
plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$");

## Double Well Example

Now, we try this same method on data from Molecular Dynamics simulations. We are trying to generate samples based on a double well MD simulation that has two potential wells that look like this:

<img src="images/doublewell.png" width=450 height=175>

It is often difficult to start a simulation in one well and reach the other because the potential exponentially climbs in between, so we start first by generating samples from the two wells and then the network from that data.

In [ ]:
def double_well_example():
    sim1 = DoubleWellSim("configs/double_well_config_1.yml")
    sim2 = DoubleWellSim("configs/double_well_config_2.yml")    
    sim1.runSimulation()
    sim2.runSimulation()
    a = np.array(sim1.coordinate_logger.coordinates, dtype=np.float32).squeeze()
    b = np.array(sim2.coordinate_logger.coordinates, dtype=np.float32).squeeze()
    data = np.concatenate((a, b), axis=0)
    
    # we need a function to project our samples for the energy function
    fx = lambda x: x[0]
    loss = getLoss().ml_loss(simulation=sim1)
    opt = getOpt().rmsprop(1e-3)
    model = RealNVP(loss, opt, model_name='double_well')
    model = LogLoss(model)
    model = LogTargetPlot(model, simulation=sim1, xlim=[-3.8,3.8], ylim=[-4.5,4.5])
    model = FreeEnergyPlot(model, simulation=sim1, RC_func=fx)
    model = Checkpointing(model)
    trainer = Trainer(model, data)
    trainer.train(25)
    
    return trainer

In [ ]:
trained_dw = double_well_example()

In [ ]:
samples = trained_dw.model.forward_sample(10000).numpy().T
sim = DoubleWellSim("configs/double_well_config_1.yml")
plt.figure(figsize=(9,6))
plt.title("Samples from Double Moon Trained Model")
plot_2D_potential(sim.central_potential, xlim=[-3.8,3.8], ylim=[-4.5,4.5], cmap='jet')
plt.scatter(samples[0], samples[1], s=1.15, c='white', cmap='jet')
plt.xlim([-3.8,3.8])
plt.ylim([-4.5,4.5])
plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$");

## Mueller Potential Well
Now we will try and generate samples from another molecular dynamics simualtion, the mueller potential. This potential has three distinct wells given by the following distribution:

<img src="images/muellerpotential.png" width=450 height=175>

Because of the increased difficulty, we use a different loss function for training.

In [ ]:
def mueller_potential_example():
    sim1 = MuellerWellSim("configs/muller_well_config_1.yml")
    sim2 = MuellerWellSim("configs/muller_well_config_2.yml")
    sim3 = MuellerWellSim("configs/muller_well_config_3.yml")
    sim1.runSimulation()
    sim2.runSimulation()
    sim3.runSimulation()
    
    a = np.array(sim1.coordinate_logger.coordinates, dtype=np.float32).squeeze()
    b = np.array(sim2.coordinate_logger.coordinates, dtype=np.float32).squeeze()
    c = np.array(sim3.coordinate_logger.coordinates, dtype=np.float32).squeeze()
    data = np.concatenate((a, b, c), axis=0)

    # a different reaction coordinate projection
    fx = lambda x: np.dot(x, np.array([1, -1]))/np.dot(np.array([1,-1]),np.array([1,-1]))
    
    loss = getLoss().rc_kl_loss(simulation=sim1, rc_func = fx, vmin = 0, vmax=1.5, turnover=100)
    opt = getOpt().rmsprop(5e-5)
    model = RealNVP(loss, opt, model_name="mueller_well", loc=[-0.25, 0.75], scale=[1.0, 1.0])
    model = LogLoss(model)
    model = LogTargetPlot(model, simulation=sim1, xlim=[-1.5, 1.1], ylim=[-0.5, 2.])
    model = FreeEnergyPlot(model, simulation=sim1, RC_func=fx)
    trainer = Trainer(model, data)
    trainer.train(40)
    
    return trainer

In [ ]:
trained_mp = mueller_potential_example()

In [ ]:
samples = trained_mp.model.forward_sample(10000).numpy().T
sim = MuellerWellSim("configs/muller_well_config_1.yml")
plt.figure(figsize=(12,9))
plt.title("Samples from Mueller Potential Trained Model")
plot_2D_potential(sim.central_potential, xlim=[-1.5, 1.1], ylim=[-0.5, 2.], cmap='jet')
plt.scatter(samples[0], samples[1], s=1.15, c='white', cmap='jet')
plt.xlim([-1.5, 1.1])
plt.ylim([-0.5, 2.])
plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$");

## Dimer Solution Example

We are moving from the 2D space and moving into a 72 dimensional space. We have 34 solvent particles and 2 dimer particles each with an x and y coordinate; 36 particles with two dimensions each equaling 72 complete dimensions. Below we have plotted each of the particles in the beginning of the solution. 

<img src="images/dimer.png" height=175 width=450 />


We can see how the dimer reacts within the solvent in the below video:

<video controls src="images/dimer_vid.mp4" height=350 width=900 />

In [12]:
def dimer_example():
    dimer_sim_md = SimulationData("configs/dimer_sim_config_md_1.yml")
    dimer_sim_md.loadSimulation("data/dimer_md_traj.npy")
    
    data = dimer_sim_md.getData().reshape((-1, 72)).astype(np.float32)
    fx = lambda x: np.linalg.norm(np.array(x[0:2]) - np.array(x[2:4]))
    loss = getLoss().rc_kl_loss(dimer_sim_md, fx, .5, 2.5, ndims=72)
    opt = getOpt().rmsprop(1e-4)
    model = RealNVP(loss, opt, in_shape=[data.shape[1]], chain_length=8, nn_layers=[200, 200, 200], loc=[0.]*72, scale=[1.]*72, model_name='dimer')
    model = LogLoss(model)
#     model = FreeEnergyPlot(model, dimer_sim_md, fx, reshape=(36,2))
    model = DimerAverageLocationPlot(model)
    trainer = Trainer(model, data)
    trainer.train(20)
    return trainer

In [13]:
trained_dm = dimer_example()

../core/network_logging.py:134: RuntimeWarning: overflow encountered in exp
  weights.append(np.exp(-self.simulation.getEnergy(t) + lp))
../core/network_logging.py:138: RuntimeWarning: invalid value encountered in true_divide
  probs = (counts / np.sum(counts)) + 1e-9
../core/network_logging.py:142: RuntimeWarning: invalid value encountered in greater
  plt.plot(bin_centers[FE > -np.log(1e-9)], FE[FE > -np.log(1e-9)])
../core/network_logging.py:134: RuntimeWarning: overflow encountered in exp
  weights.append(np.exp(-self.simulation.getEnergy(t) + lp))
../core/network_logging.py:138: RuntimeWarning: invalid value encountered in true_divide
  probs = (counts / np.sum(counts)) + 1e-9
../core/network_logging.py:142: RuntimeWarning: invalid value encountered in greater
  plt.plot(bin_centers[FE > -np.log(1e-9)], FE[FE > -np.log(1e-9)])


KeyboardInterrupt: 